Classification of cuisines based on  the ingredients used for preparing the cuisine using neural networks

Importing the required libraries

In [ ]:
from google.colab import files , drive
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder

mounting google drive to colab

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


file paths to the .json data

In [ ]:
file_1 = 'gdrive/My Drive/Colab Notebooks/kaggle_ML/cooking/train.json'
file_2 = 'gdrive/My Drive/Colab Notebooks/kaggle_ML/cooking/test.json'


reading .json format files to dataframe

In [ ]:
train = pd.read_json(file_1)
test = pd.read_json(file_2)

In [ ]:
id=test['id']

In [ ]:
n=train.shape[0]
n

39774

number of different types of cuisines

In [ ]:
train['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [ ]:
train.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


Finding columns with NaN entries

In [ ]:
print(test.isnull().sum().sort_values(ascending=False))
train.isnull().sum().sort_values(ascending=False)

ingredients    0
id             0
dtype: int64


ingredients    0
cuisine        0
id             0
dtype: int64

concatinating both training and test data

In [ ]:
total = pd.concat((train,test),ignore_index=True)
total.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


making a list of total ingredients in the dataset

In [ ]:
ingredient = set()
for index in range(total.shape[0]):
  ings = total['ingredients'][index]
  for ing in ings:
    ingredient.add(ing)

columns =list(ingredient)

In [ ]:
len(columns)

7137

One hot encoding of the ingredients\\
As the datasets are very large for loop is used

In [ ]:
j=0
for col in columns:
  total[col] = 0
  for i in range(total.shape[0]):
    if col in total['ingredients'][i]:
      total[col][i]=1
  j=j+1
  print(j,col)
  

In [ ]:
total.head()

downloading the datasets in .csv format to load later to save memory

In [ ]:
total.to_csv('total.csv')
files.download('total.csv')

Retaining the preprocessed train and test sets

In [ ]:
final_train = total[:39774]
final_test = total[39774:]
final_train.drop(['id','Unnamed: 0','ingredients'],1,inplace=True)
final_test.drop(['id','Unnamed: 0','cuisine','ingredients'],1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#le = LabelEncoder()
#final_train['cuisine']=le.fit_transform(final_train['cuisine'])

In [ ]:
final_train.to_csv('ftrain.csv')
final_test.to_csv('ftest.csv')
files.download('ftrain.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('ftest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
file_path1= 'gdrive/My Drive/Colab Notebooks/kaggle_ML/cooking/ftrain.csv'
file_path2= 'gdrive/My Drive/Colab Notebooks/kaggle_ML/cooking/ftest.csv'
final_train = pd.read_csv(file_path1)
final_test = pd.read_csv(file_path2)

In [ ]:
final_test.shape

(9944, 7138)

In [ ]:
final_train.shape

(39774, 7139)

one hot encoding of the cuisines

In [ ]:
asdf = pd.get_dummies(final_train['cuisine'])

Dropping the cuisines column

In [ ]:
final_train.drop(['Unnamed: 0','cuisine'],1,inplace=True)
final_test.drop(['Unnamed: 0'],1,inplace=True)

splitting for train and validation

In [ ]:
df_target = asdf[:35000]
val_tar = asdf[35000:]

In [ ]:
val = final_train[35000:]
final_train=final_train[:35000]

In [ ]:
final_train.shape

(35000, 7137)

In [ ]:
final_test.shape

(9944, 7137)

As i am using tensorflow ,converting the dataframe to tensors

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((final_train.values,df_target.values))
train=dataset.shuffle(len(final_train)).batch(32)

In [ ]:
val_set = (tf.data.Dataset.from_tensor_slices((val.values,val_tar.values))).shuffle(len(val)).batch(32)

In [ ]:
test_set = (tf.data.Dataset.from_tensor_slices(final_test.values)).shuffle(len(final_test)).batch(32)

In [ ]:
train

<BatchDataset shapes: ((None, 7137), (None, 20)), types: (tf.int64, tf.uint8)>

nueral network model

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(7137))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(1024))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(256))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(20))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Softmax())

loss functions and metric

In [ ]:
model.compile(optimizer='sgd',
                loss=tf.keras.losses.CategoricalCrossentropy(True),
                metrics=['accuracy'])

training

In [ ]:
model.fit(train,epochs=50,validation_data=val_set)

Epoch 1/50
1094/1094 [==============================] - 22s 20ms/step - loss: 2.8994 - accuracy: 0.2753 - val_loss: 2.7479 - val_accuracy: 0.5354
Epoch 2/50
1094/1094 [==============================] - 17s 15ms/step - loss: 2.7529 - accuracy: 0.4992 - val_loss: 2.6452 - val_accuracy: 0.6154
Epoch 3/50
1094/1094 [==============================] - 17s 16ms/step - loss: 2.6588 - accuracy: 0.5807 - val_loss: 2.5686 - val_accuracy: 0.6542
Epoch 4/50
1094/1094 [==============================] - 17s 15ms/step - loss: 2.5907 - accuracy: 0.6268 - val_loss: 2.5069 - val_accuracy: 0.6816
Epoch 5/50
1094/1094 [==============================] - 16s 15ms/step - loss: 2.5400 - accuracy: 0.6535 - val_loss: 2.4649 - val_accuracy: 0.6965
Epoch 6/50
1094/1094 [==============================] - 13s 12ms/step - loss: 2.5005 - accuracy: 0.6757 - val_loss: 2.4371 - val_accuracy: 0.7088
Epoch 7/50
1094/1094 [==============================] - 17s 15ms/step - loss: 2.4721 - accuracy: 0.6937 - val_loss: 2.4141 -

Saving weights

In [ ]:
filepath4 = 'gdrive/My Drive/Colab Notebooks/kaggle_ML/cooking/Models/weights'
filepath3 = 'gdrive/My Drive/Colab Notebooks/kaggle_ML/cooking/Models'
model.save_weights(filepath4)
model.save(filepath3)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: gdrive/My Drive/Colab Notebooks/kaggle_ML/cooking/Models/assets


predicting the probabilty of classes of test set

In [ ]:
answers = model.predict(test_set)

In [ ]:
answers.shape

(9944, 20)

classification of the predicted probabilities to classes with highest probability

In [ ]:
pred=np.argmax(answers,1)
feat=asdf.columns.to_list()
a=[]
for i in range(9944):
  a.append(feat[pred[i]])

creating a dataset for submission

In [ ]:
sub = pd.DataFrame({'id':id,'cuisine':a})

In [ ]:
sub.reset_index(inplace=True,drop=True)


In [ ]:
feat[17]

'spanish'

In [ ]:
pred

array([17,  2, 13, ..., 16,  5, 14])

In [ ]:
sub.head()

,id,cuisine
0,18009,spanish
1,28583,cajun_creole
2,41580,mexican
3,29752,italian
4,35687,japanese


In [ ]:
sub.to_csv('sub.csv')
files.download('sub.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sdf=pd.DataFrame({'a':[1,2,3,4,5],'b':[6,7,8,9,10]})
sdf.head()

,a,b
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [ ]:
np.argmax(arr,axis=0)

array([1, 1, 0, 0, 1, 0])

In [ ]:
pred=np.argmax(answers,1)
pred.shape

(9944,)

In [ ]:
sub.head()

,id,cuisine
